## Sleep data analysis

You'll learn how to analyze sleep EEG data.

This demo is a jupyter notebook, i.e. intended to be run step by step.

Author: Eric Einspänner
<br>
Contributor: Nastaran Takmilhomayouni

First version: 6th of July 2023 ?


Copyright 2023 Clinic of Neuroradiology, Magdeburg, Germany

License: Apache-2.0

*This notebook is inspired by: https://mne.tools/stable/auto_tutorials/clinical/60_sleep.html*

*Dataset: https://ieeexplore.ieee.org/document/867928*

0. [Initial Set-Up for Google Colab](#initial-set-up-for-google-colab)
1. [Initial Set-Up (offline)](#initial-set-up-offline)
2. [Data Analysis](#Data-Analysis)
    - [Extract_30s_events_from_annotations](#Extract-30s-events-from-annotations)
    - [Create_Epochs](#Create-Epochs)


## Initial Set-Up for Google Colab
<u> Execute these code blocks just in Google Colab! </u>

In [ ]:
!wget -q -O - https://github.com/University-Clinic-of-Neuroradiology/python-bootcamp/archive/refs/heads/main.tar.gz | tar -xzf - --strip-components=2 python-bootcamp-main/notebooks/DataManagement

In [ ]:
import os
import sys
from google.colab import output
output.enable_custom_widget_manager()

sys.path.insert(0,'DataManagement')
os.chdir(sys.path[0])

In [ ]:
%pip install -q ipympl matplotlib mne

In [ ]:
import matplotlib.pyplot as plt

import mne
from mne.datasets.sleep_physionet.age import fetch_data

## Initial Set-Up (offline)

In [ ]:
# Make sure figures appears inline and animations works
# Edit this to ""%matplotlib notebook" when using the "classic" jupyter notebook interface
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt

import mne
from mne.datasets.sleep_physionet.age import fetch_data

## --- Start notebook ---

## Data Analysis

In [ ]:
# load two data files of two subjects 
ALICE, BOB = 0, 1

[alice_files, bob_files] = fetch_data(subjects=[ALICE, BOB], recording=[1])

# two files containing SC4001E0-PSG.edf and # SC4001EC-Hypnogram.edf
print(alice_files) 

# read one file from SC4001E0-PSG.edf
raw_train = mne.io.read_raw_edf(
    alice_files[0], stim_channel="Event marker", infer_types=True, preload=True
)

# read annotations from SC4001EC-Hypnogram.edf
annot_train = mne.read_annotations(alice_files[1])
raw_train.set_annotations(annot_train, emit_warning=False)

print('Information of the Sleep dataset', raw_train.info)

In [ ]:
# plot some data
# scalings were chosen manually to allow for simultaneous visualization of
# different channel types in this specific dataset
raw_train.plot(
    start = 60,
    duration = 60,
    scalings = dict(eeg=1e-4, resp=1e3, eog=1e-4, emg=1e-7, misc=1e-1),
)

### Extract 30s events from annotations

The Sleep Physionet dataset is annotated using 8 labels:

- Wake (W),
- Stage 1, Stage 2, Stage 3, Stage 4 corresponding to the range from light sleep to deep sleep,
- REM sleep (R) where REM is the abbreviation for Rapid Eye Movement sleep, movement (M), 
- and Stage (?) for any none scored segment.

We will work only with 5 stages: Wake (W), Stage 1, Stage 2, Stage 3/4, and REM sleep (R).

In [ ]:
annotation_desc_2_event_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3": 4,
    "Sleep stage 4": 4,
    "Sleep stage R": 5,
}

# keep last 30-min wake events before sleep and first 30-min wake events after
# sleep and redefine annotations on raw data
annot_train.crop(annot_train[1]["onset"] - 30 * 60, annot_train[-2]["onset"] + 30 * 60)
raw_train.set_annotations(annot_train, emit_warning=False)

#select which events are we interested in and
events_train, _ = mne.events_from_annotations(
    raw_train, event_id=annotation_desc_2_event_id, chunk_duration=30.0
)

# create a new event_id that unifies stages 3 and 4
event_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3/4": 4,
    "Sleep stage R": 5,
}

# plot events
fig = mne.viz.plot_events(
    events_train,
    event_id=event_id,
    sfreq=raw_train.info["sfreq"],
    first_samp=events_train[0, 0],
)

# keep the color-code for further plotting
stage_colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

### Create Epochs

In [ ]:
# Create Epochs from the data based on the events found in the annotations
tmax = 30.0 - 1.0 / raw_train.info["sfreq"]  # tmax in included , number od samples

epochs_train = mne.Epochs(
    raw=raw_train,
    events=events_train,
    event_id=event_id,
    tmin=0.0,
    tmax=tmax,
    baseline=None,
)
del raw_train # we can delete raw_train object and continue with epochs_train object

print(epochs_train) 

In [ ]:
# test everything for the other subject, bob !
raw_test = mne.io.read_raw_edf(
    bob_files[0], stim_channel="Event marker", infer_types=True, preload=True
)
annot_test = mne.read_annotations(bob_files[1])
annot_test.crop(annot_test[1]["onset"] - 30 * 60, annot_test[-2]["onset"] + 30 * 60)
raw_test.set_annotations(annot_test, emit_warning=False)
events_test, _ = mne.events_from_annotations(
    raw_test, event_id=annotation_desc_2_event_id, chunk_duration=30.0
)
epochs_test = mne.Epochs(
    raw=raw_test,
    events=events_test,
    event_id=event_id,
    tmin=0.0,
    tmax=tmax,
    baseline=None,
)
del raw_test

print(epochs_test)